In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# 전체 데이터셋 불러오기

In [3]:
data_2403 = np.load("../data/data_/data_2403.npy")

# 이미지 전처리 알고리즘

이미지 나누기

In [2]:
def split_images(image_files, p=15, input_length=14):
    input_images = []
    output_images = []
    r = len(image_files)//p if(len(image_files)%p==0) else len(image_files)//p + 1
    for repeats in range(r):
        # rp 배열에 p단위수로 끊어서 넣기
        i_rp = image_files[:p][:input_length]
        o_rp = image_files[:p][input_length:]
        # 기존 배열에서 앞부분은 삭제하면서 당기기
        image_files = image_files[p:]
        # 배열에 삽입 해버렷
        input_images.append(i_rp)
        output_images.append(o_rp)
        # 버렷!
        if len(image_files) < p:
            break
    # 넘파이 배열로 전환 후 내보내기
    return np.array(input_images), np.array(output_images)

이미지 리사이즈

In [4]:
def resize_images(dataset, target_size):
    resized_dataset = tf.image.resize(dataset, target_size, method='bilinear')
    return resized_dataset

# 본격적인 이미지 전처리 후 저장

쪼개기1

In [5]:
np.save("../data/data_/data_2403_300",resize_images(data_2403[:300], (40,40)))

쪼개기2

In [6]:
np.save("../data/data_/data_2403_715",resize_images(data_2403[300:], (40,40)))

쪼갠 거 합친 후 다시 입력 출력으로 나누기

In [13]:
a = "15"
data_300 = np.load("data/data_/data_24"+a+"_300.npy")
data_715 = np.load("data/data_/data_24"+a+"_715.npy")
data_ = np.concatenate((data_300, data_715), axis=0)
inputs, outputs = split_images(data_)
print(inputs.shape, outputs.shape)
np.save("data/data_/data_24"+a+"_input",inputs)
np.save("data/data_/data_24"+a+"_output",outputs)

(47, 14, 40, 40, 4) (47, 1, 40, 40, 4)


In [22]:
input = np.load("data/data_/data_2404_input.npy")
output = np.load("data/data_/data_2404_output.npy")
input1 = np.load("data/data_/data_2405_input.npy")
output1 = np.load("data/data_/data_2405_output.npy")
input2 = np.load("data/data_/data_2406_input.npy")
data_ = np.concatenate((input, input1, input2), axis=0)
data_.shape

(141, 14, 40, 40, 4)

In [39]:
inputs = np.zeros((0,14,40,40,4))
outputs = np.zeros((0,1,40,40,4))
for i in range(12):
    a = "0"+str(i+4) if((i+4)<10) else str(i+4)
    input = np.load("data/data_/data_24"+a+"_input.npy")
    output = np.load("data/data_/data_24"+a+"_output.npy")
    inputs = np.concatenate((inputs, input), axis=0)
    outputs = np.concatenate((outputs, output), axis=0)
print(inputs.shape, outputs.shape)

(564, 14, 40, 40, 4) (564, 1, 40, 40, 4)


In [41]:
np.save("data/data_/data_total_inputs",inputs)
np.save("data/data_/data_total_outputs",outputs)

# 1시간 간격으로 데이터 다시 처리해야해!!!

데이터 다시 수합

In [3]:
all_data = np.zeros((0,40,40,4))
for i in range(12):
    a = "0"+str(i+4) if((i+4)<10) else str(i+4)
    data_300 = np.load(f"data/data_/data_24{a}_300.npy")
    data_715 = np.load(f"data/data_/data_24{a}_715.npy")
    data_ = np.concatenate((data_300, data_715), axis=0)
    all_data = np.concatenate((all_data, data_),axis=0)
print(all_data.shape)

(8575, 40, 40, 4)


이미지 1시간 간격으로 일단 가져오기

In [4]:
split_data_ = np.zeros((0,40,40,4))
for i in range(0,len(all_data),30):
    split_data_ = np.concatenate((split_data_,all_data[i].reshape((1,40,40,4))), axis=0)
print(split_data_.shape)

(286, 40, 40, 4)


In [5]:
def split_images(image_files, p=15, input_length=14):
    input_images = []
    output_images = []
    r = len(image_files)//p if(len(image_files)%p==0) else len(image_files)//p + 1
    for repeats in range(r):
        # rp 배열에 p단위수로 끊어서 넣기
        i_rp = image_files[:p][:input_length]
        o_rp = image_files[:p][input_length:]
        # 기존 배열에서 앞부분은 삭제하면서 당기기
        image_files = image_files[p:]
        # 배열에 삽입 해버렷
        input_images.append(i_rp)
        output_images.append(o_rp)
        # 버렷!
        if len(image_files) < p:
            break
    # 넘파이 배열로 전환 후 내보내기
    return np.array(input_images), np.array(output_images)

원래는 split_images로 하려했는데 데이터가 너무 적어보여서 split_data로 부풀렸음

In [6]:
def split_data(data, gap, total_length, input_length):
    input = []
    output = []
    i = 0
    while i+total_length+1 <= len(data):
        input.append(data[i:total_length+i][:input_length])
        output.append(data[i:total_length+i][input_length:])
        i += gap
    return np.array(input), np.array(output)

In [11]:
inputs, outputs = split_data(split_data_, gap=1, total_length=12, input_length=11)

In [12]:
inputs.shape, outputs.shape

((274, 11, 40, 40, 4), (274, 1, 40, 40, 4))

In [48]:
np.save("data/data_/data_1hours_inputs", inputs)
np.save("data/data_/data_1hours_outputs", outputs)